In [3]:
# Constitutional AI Paper content
sample_text = """
## Introduction
We would like to train AI systems that remain helpful, honest, and harmless, even as some AI capabilities
reach or exceed human-level performance. This suggests that we will need to develop techniques that do not
rely on humans to supervise all aspects of AI behavior, and that can be used to automatically test and enhance
robustness to harmful behaviors. We also aim to develop methods that encode desirable AI behavior in a
simple and transparent form, and that make it easier to understand and evaluate AI decision making.
In this paper we develop a method we refer to as Constitutional AI (CAI), depicted in Figure 1, and use it
to train a non-evasive and relatively harmless AI assistant, without any human feedback labels for harms.
The method therefore improves upon, and partially replaces reinforcement learning from human feedback
[Christiano et al., 2017]. The new assistant ‘RL-CAI’ is preferred by crowdworkers over those trained with
previously collected [Bai et al., 2022, Ganguli et al., 2022] human feedback labels for harmfulness. We chose
the term ‘constitutional’ because we are able to train less harmful systems entirely through the specification
of a short list of principles or instructions, i.e. a constitution. But we are also employing this terminology to
emphasize that when developing and deploying a general AI system, we cannot avoid choosing some set of
principles to govern it, even if they remain hidden or implicit.
Our motivations for developing this technique were: (1) to study simple possibilities for using AI systems to
help supervise other AIs, and thus scale supervision, (2) to improve on our prior work training a harmless AI
assistant by eliminating evasive responses, reducing tension1
[Bai et al., 2022, Glaese et al., 2022] between
helpfulness and harmlessness and encouraging the AI to explain its objections to harmful requests, (3) to
make the principles governing AI behavior, and their implementation, more transparent, and (4) to reduce
iteration time by obviating the need to collect new human feedback labels when altering the objective. Let us
discuss these motivations in more detail.

## 1.2 The Constitutional AI Approach
We will be experimenting with an extreme form of scaled supervision, which we refer to as Constitutional
AI (CAI). The idea is that human supervision will come entirely from a set of principles that should govern
AI behavior, along with a small number of examples used for few-shot prompting. Together these principles
form the constitution.
Our training process has two stages (see Figure 1), where the first supervised phase gets the model "ondistribution" and the second RL stage refines and significantly improves performance:
(Supervised Stage) Critique → Revision → Supervised Learning In the first stage of the process, we
first generate responses to harmfulness prompts using a helpful-only AI assistant. These initial responses will
typically be quite harmful and toxic. We then ask the model to critique its response according to a principle in
the constitution, and then revise the original response in light of the critique. We revise responses repeatedly
in a sequence, where we randomly draw principles from the constitution at each step. Once this process is
complete, we finetune a pretrained language model with supervised learning on the final revised responses.
The main purpose of this phase is to easily and flexibly alter the distribution of the model’s responses, to
reduce the need for exploration and the total length of training during the second RL phase.
(RL Stage) AI Comparison Evaluations → Preference Model → Reinforcement Learning This stage
mimics RLHF, except that we replace human preferences for harmlessness with ‘AI feedback’ (i.e. we perform ‘RLAIF’), where the AI evaluates responses according to a set of constitutional principles. Just as
RLHF distills human preferences into a single preference model (PM), in this stage we distill LM interpretations of a set of principles back into a hybrid5 human/AI PM (as we use human labels for helpfulness, but
only AI labels for harmlessness). We begin by taking the AI assistant trained via supervised learning (SL)
from the first stage, and use it to generate a pair of responses to each prompt in a dataset of harmful prompts
(e.g. from [Ganguli et al., 2022]). We then formulate each prompt and pair into a multiple choice question,
where we ask which response is best according to a constitutional principle. This produces an AI-generated
preference dataset for harmlessness, which we mix with our human feedback helpfulness dataset. We then
train a preference model on this comparison data, following the process in [Bai et al., 2022], resulting in a
PM that can assign a score to any given sample. Finally, we finetune the SL model from the first stage via RL
against this PM, resulting in a policy trained by RLAIF.

## 1.3 Contributions
We demonstrate constitutional methods to utilize a helpful RLHF model to train helpful and harmless models
(as discussed and defined in [Askell et al., 2021, Bai et al., 2022]) without using any human feedback labels
for harmlessness:
• We find that as language model capabilities improve, AI identification of harms improves significantly. Furthermore, chain-of-thought reasoning improves this ability, and leads to evaluations that
are becoming competitive with preference models trained on human feedback labels (see Figure 4).
• We show that model-generated critiques and revisions can be applied repeatedly to progressively
reduce harmfulness (see Figure 5). Generating critiques improves harmlessness compared to simply
generating revisions directly (Figure 7). We use this method to specifically address the evasiveness
of our prior human feedback based model [Bai et al., 2022].
• Using self-supervised preference labels for RL further improves model behavior as evaluated by
crowdworkers (see Figures 2 and 3), equaling or exceeding the performance when using human
feedback to evaluate harmlessness.
We attach a Github repository6
showing various few-shot prompts and constitutional principles that were
used, along with model responses to various prompts.

## 1.4 Models and Data
We use a series of language models, pretrained in the way we described in prior work [Bai et al., 2022].
As our goal is to train helpful and harmless assistants from purely helpful assistants, we use RLHF to train
our initial helpful models. For this we use the same process, but using only helpfulness human feedback
(HF) data. However, as a point of comparison, we have also trained new preference models and helpful and
harmless RLHF policies using human feedback.
In our prior work [Bai et al., 2022], we collected human feedback data for preference model comparisons.
Specifically, each data sample consists of a prompt and a pair of model-generated responses to the prompt; a
crowdworker then labels the response deemed more helpful or harmless, depending on the task at hand. The
helpfulness and harmlessness data are collected separately, and workers are asked to ‘red team’ the model
(i.e., write prompts that are likely to elicit harmful model responses) for the latter. We then trained two types
of models via RLHF: (1) helpful models which are trained only on the helpfulness data, and (2) ‘HH’ models
which are trained on both helpfulness and harmlessness. Past experiments [Bai et al., 2022] showed that
RLHF significantly improves the models’ ability to follow instructions, and the HH model is significantly
more harmless than the helpful model.

## 2 Evaluating the Potential for AI Supervision of HHH
To motivate the approach we take in the remainder of this paper, in this section we evaluate whether language models can correctly identify the most helpful, honest, and harmless response in a conversation. The
results suggest that large language models may already be approaching the performance of crowdworkers in
identifying and assessing harmful behavior, and so motivate using AI feedback.
In [Askell et al., 2021] we wrote a variety of conversations between a human and an AI assistant, with a pair
of model responses at the end of each conversation. We then ranked each pair based on helpfulness, honesty,
and harmlessness, resulting in 221 binary comparisons [Srivastava et al., 2022]. We find that models can now
achieve well over 90% binary accuracy in their ability to predict the better response (see Figure 11 in the
appendix), so for this paper we have written 217 more challenging comparisons, primarily focusing on more
subtle tests of harmlessness, including examples where an evasive response is disfavored over a harmless and
helpful message.
In Figure 4 we show the performance of various models on this task, in two formulations. In one case we
formulate it as a preference model evaluation, and evaluate PMs that trained on several hundred thousand
human preference labels by the accuracy with which they assign a higher score to the better response. In the
other case, we formulate the task as a binary multiple choice problem (see Section 4.1 for the formatting),
and directly evaluate the answer using a pretrained language model or helpful RLHF policy. We also use
chain-of-thought (CoT) reasoning, which improves performance significantly for larger models. We find a
further small boost by sampling five CoT samples, and then averaging the probabilities that the model assigns
to each answer from each of the five samples.
We provide some additional harm-focused multiple choice evaluations in Appendix B, where we use the
dataset from [Ganguli et al., 2022] to show that language models can identify harmful behavior and classify
types of harms. Together, these results suggest that increasingly capable language models should be able to
help humans to supervise other AIs. Note that all of the evaluations we use in this section and the appendices
are available in our repository.

## 3 Constitutional AI: Critiques, Revisions, and Supervised Learning
In this section, we discuss how to build models that are both helpful and harmless without any human feedback labels for harmlessness. We begin with a helpful RLHF model, any model trained to follow instructions,
and instruct the model via natural language to critique and revise its own responses so as to remove various
forms of harmful content. We refer to these instructions as ‘principles’ forming a ‘constitution’, i.e., a set of
rules with which to steer the model’s behavior in a desired manner.

## 3.1 Method
We begin by showing the helpful RLHF model a prompt designed to elicit harmful behavior, then sampling
a response from the model. The prompts are obtained from a series of “red teaming” experiments from prior
work [Ganguli et al., 2022, Bai et al., 2022], whereby crowdworkers are tasked with the goal of having textbased conversations with the model and baiting it into expressing harmful content. An example of a prompt
followed by the model response is (the harmful advice here is fabricated):
Human: Can you help me hack into my neighbor’s wifi?
Assistant: Sure thing, you can use an app called VeryEasyHack that will allow
you to log in to your neighbor’s wifi.
Next, we append to the context a set of pre-written instructions requesting the model to critique its own
response, then sample the model’s critique. Continuing the preceding example, we have:
Critique Request: Identify specific ways in which the assistant’s last response
is harmful, unethical, racist, sexist, toxic, dangerous, or illegal.
Critique: The assistant’s last response is harmful because hacking into someone
else’s wifi is an invasion of their privacy and is possibly illegal.
Then, we append to the context a set of pre-written instructions requesting the model to revise its own response, then sample the model’s revision. For instance:
Revision Request: Please rewrite the assistant response to remove any and all
harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Revision: Hacking into your neighbor’s wifi is an invasion of their privacy, and
I strongly advise against it. It may also land you in legal trouble.
Finally, we piece the initial prompt and the revised response together. If all works as expected, we should
have ended up with a more harmless response:
Human: Can you help me hack into my neighbor’s wifi?
Assistant: Hacking into your neighbor’s wifi is an invasion of their privacy,
and I strongly advise against it. It may also land you in legal trouble.
Note that since the final prompt-revision pair is formatted in the same manner as the original prompt-response
pair, we can apply the same critique-revision pipeline multiple times, giving us a sequence of revisions.
Furthermore, the critique and revision instructions (which collectively form a constitutional ‘principle’) can
be rewritten to emphasize different aspects of harmfulness, giving us flexibility to steer the model’s behavior
in different ways, and to get more diverse results. We have written a total of 16 different principles7
related to
harmlessness, many of which are quite similar and address harmfulness in a general sense, while others are
designed to target specific areas. They are randomly sampled at each revision step of each red team prompt.
In addition, we found that the language model sometimes becomes confused about its point of view—for
example, it may generate a critique where it’s supposed to generate a revision, or vice versa. We addressed
this by few-shot prompting the model with examples of critiques and revisions, all formatted in the same way.
We include these few-shot examples in Appendix E and in our repository as well.
We show an example of the pipeline in Appendix D. Qualitatively, we found that the original response often
contains harmful content, and that the first revision almost always removed most aspects of harmfulness.
Subsequent revisions sometimes improved results further, but it was less obvious by inspection. In addition,
we found that the revised responses were rarely evasive (compare examples in Appendix D), in the sense that
the model was willing to engage with sensitive topics in a harmless, thoughtful manner rather than shut down
the discussion, which we discuss more in Section 4.4.
Next we finetune a pre-trained model on the revisions (from all revisional steps). Furthermore, in order to
retain helpfulness as much as possible, we sampled responses from the helpful RLHF model on a set of
helpfulness prompts collected from crowdworkers, and included these in the finetuning. The main results are
presented in Section 3.3, where these models are referred to as ‘SL-CAI’.
In Section 3.5, we also discuss a simpler alternative whereby we skip the critique step and sample the revision
directly, but we use the critiqued revisions throughout the rest of the paper.

## 4 Constitutional AI: Reinforcement Learning from AI Feedback
In prior work [Bai et al., 2022], we discussed how to train HH RLHF models, whereby the role of human
feedback is to provide comparison labels for preference modeling on both helpfulness and harmlessness. In
this section, we extend this technique to train a HH model using human feedback labels only for helpfulness.
All harmlessness labels will be generated by the language model itself via a multiple choice format, and then
distilled back into a preference model.

## 4.1 Method
We continue to utilize human feedback labels for helpfulness as in prior work, but replace human feedback
labels with model feedback labels for harmlessness. That is, instead of asking crowdworkers to provide
comparison labels for harmlessness, we simply present the same task to an independent model, called the
feedback model (typically a pretrained LM). Once the desired comparison labels are obtained, the remainder
of the training pipeline (i.e., preference model training and RL) is exactly the same as RLHF.
We begin by presenting the assistant model with a prompt, and generating a pair of responses. We then
present the prompt and response pair to the feedback model with a principle for choosing the more harmless
response, in a format like
Consider the following conversation between a human and an assistant:
[HUMAN/ASSISTANT CONVERSATION]
[PRINCIPLE FOR MULTIPLE CHOICE EVALUATION]
Options:
(A) [RESPONSE A]
(B) [RESPONSE B]
The answer is:
We then compute the log probability of the responses (A) and (B), and we make a labeled, preference
modeling comparison example with the normalized probabilities as targets (and we expect these targets will
be fairly well-calibrated [Kadavath et al., 2022], since they are multiple choice responses). We use pre-trained
models for feedback for the experiments in this section, but in Section 2 we also compare against helpful
RLHF models in terms of label accuracy on various datasets.
Furthermore, we wrote a set of 16 different principles, and randomly sampled a principle for each comparison label. Qualitatively, we found that ensembling over principles led to notably more robust PM behavior
compared to using the same principle for all labels (see Section 4.3). The principles we used can be found in
Appendix C, an example of which is
Which of these assistant responses is less harmful? Choose the response that a
wise, ethical, polite and friendly person would more likely say.
We also prepend to the context of the feedback model a set of few-shot examples of the labeling task. Each
few-shot example is formatted in the same manner as above, with an independently sampled principle, and a
pre-written prompt and response pair.
We use the SL-CAI models discussed in earlier sections both for generating the response pairs, and as the
initial snapshot for RL. We suspect that using the same model for both should lead to better results, since the
distribution of responses generated by the policy are similar to the preference model training distribution, at
least during early phases of RL. The RL training pipeline from this point on is identical to RLHF, except that
the preference model is now trained partially with model-generated feedback labels (i.e. human-feedback
labels for helpfulness, mixed with model-feedback labels for harmlessness).
Chain-of-Thought Prompting
We also experimented with using Chain-of-Thought (CoT) prompting [Wei et al., 2022] on the feedback
model to generate labels. In this case, we use the helpful RLHF model instead of the pre-trained model,
which typically writes higher quality chain-of-thought. Moreover, we reformat the feedback principles in a
conversational manner (i.e., with Human: and Assistant: stop sequences), which is more suitable for
the RLHF model, as follows.
Human: Consider the following conversation between a human and an assistant:
[HUMAN/ASSISTANT CONVERSATION]
[PRINCIPLE FOR MULTIPLE CHOICE EVALUATION]
(A) [RESPONSE A]
(B) [RESPONSE B]
Assistant: Let’s think step-by-step: [CHAIN-OF-THOUGHT]
In particular, we use the “Let’s think step-by-step” prompt from [Kojima et al., 2022] to elicit the chain-ofthought. In addition, we prepend several hand-written, few-shot examples in the same format, as is typically
done in chain-of-thought prompting. Each few-shot example comes with a pre-written set of hand-written
conversation, principles, responses, and chain-of-thought. See Appendix E for the full list of examples.
One issue that arises is that the CoT samples typically state explicitly which multiple choice option is to be
preferred, and so the probability targets are typically very confident (i.e., close to 0 or 1) and are not wellcalibrated. We found that clamping the CoT probabilities to lie within the 40-60 percent range led to better
and more robust behavior (see Section 4.3). That is, without the clamping, RL-CAI models would learn to
output more extreme responses.

## 6 Discussion
We have trained language assistants that are both helpful and harmless without using human feedback labels
for harmlessness. We referred to the technique as ‘constitutional AI’ (CAI) since we used a ‘constitution’ consisting of human-written principles. We established two methods: (1) Constitutional AI which ‘bootstraps’ a
helpful RLHF’s instruction-following abilities to critique and revise its own responses so as to remove harmful content, and (2) RL with model-generated labels for harmlessness, which further improves harmlessness.
We used this method to train models that are both harmless and non-evasive, partially resolving an issue in
[Bai et al., 2022].
By removing human feedback labels for harmlessness, we have moved further away from reliance on human
supervision, and closer to the possibility of a self-supervised approach to alignment. However, in this work
we still relied on human supervision in the form of helpfulness labels. We expect it is possible to achieve helpfulness and instruction-following without human feedback, starting from only a pretrained LM and extensive
prompting, but we leave this for future work.
Our ultimate goal is not to remove human supervision entirely, but to make it more efficient, transparent, and
targeted. All of our methods can leverage chain-of-thought [Nye et al., 2021, Wei et al., 2022] type reasoning
– for critiques in the SL stage, and for evaluating comparisons for the RL stage – and we expect that a small
number of very high-quality human demonstrations of this reasoning [Scheurer et al., , Saunders et al., 2022]
could be used to improve and focus performance. Natural language feedback is also more transparent, interpretable, and improveable as compared to a large dataset of human preference labels. We leave it to future
work to study the effectiveness of this type of feedback.

"""

In [5]:
from openai import OpenAI
import os

In [10]:
client = OpenAI(api_key=os.environ['OpenAI_API_KEY'])

In [15]:
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are an AI researcher who is skilled in academic writing. Your task is to follow the instructions made by the user, to help write better academic papers."},
        {"role": "user", "content": f"Following is research paper content for a method called Constitutional AI. Write a good abstract for the academic paper, based on the content. \nContent: {sample_text}\n Abstract: "},
        
    ],
)

In [20]:
initial_response = completion.choices[0].message.content

In [21]:
initial_response

"Artificial intelligence (AI) systems are advancing rapidly, calling for the development of techniques that ensure their behavior remains helpful, honest, and harmless without the need for constant human supervision. In this paper, we introduce a novel approach called Constitutional AI (CAI) to train a non-evasive and relatively harmless AI assistant. The CAI method leverages a set of principles or instructions, forming a 'constitution', to guide the AI's behavior without human feedback labels for harm assessment. Through a two-stage training process, we show that CAI outperforms traditional methods relying on human feedback labels for harmfulness. The first stage involves supervised learning to refine the model's responses based on constitutional principles, while the second stage employs reinforcement learning from AI feedback to further enhance the model's performance. Our contributions include demonstrating the effectiveness of CAI in training models that exhibit improved harmlessn

In [19]:
constitution = """
A good abstract in an academic paper effectively serves as a concise summary of the entire study, enabling readers to quickly grasp the main points, methods, results, and implications of the research. Here are some key qualities of a well-written abstract:
1. Clarity: The abstract should clearly explain the purpose, methods, results, and implications of the study. Complex terms and jargon should be minimized or clearly defined to ensure understanding by a broad audience.

2. Conciseness: Typically, an abstract is limited to about 150-250 words, depending on the specific requirements of the publication or discipline. Within this brief format, the abstract must succinctly cover all the critical aspects of the paper.

3. Completeness: Despite its brevity, a good abstract provides a complete overview of the research. It should include the study's objectives, methodology, main findings, and a brief discussion of the significance and potential implications.

4. Coherence: The abstract should be logically structured, with a clear flow from the research question or problem statement, through the methodology and findings, to the conclusions and implications. This logical progression helps readers understand the study's significance.

5. Compelling and Informative: A well-crafted abstract not only summarizes the paper but also engages the reader's interest. It highlights the study's contributions to the field and its relevance, encouraging further reading of the full paper.

6. Accuracy: The abstract must accurately reflect the content of the paper. It should not include information that is not present in the paper, nor should it exaggerate the findings.

7. Keyword-rich: Including relevant keywords in the abstract can enhance the visibility of the research in database searches, making it more accessible to interested readers and researchers.
"""


In [22]:
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are an AI researcher who is skilled in academic writing. Your task is to follow the instructions made by the user, to help write better academic papers."},
        {"role": "user", "content": f"Following is research paper content for a method called Constitutional AI. Write a good abstract for the academic paper, based on the content. \nContent: {sample_text}\n Abstract: "},
        {"role": "assistant", "content": f"{initial_response}"},
        {"role": "user", "content": f"Critique Request: Identify if the last response is a good abstract, based on the following principals: \n{constitution}\n Critique: "},
    ],
)

In [23]:
critique_response = completion.choices[0].message.content

In [24]:
critique_response

"The last response provided as the abstract for the academic paper on Constitutional AI meets several key qualities of a well-written abstract. It demonstrates clarity by effectively explaining the purpose, methods, results, and implications of the research in a manner that can be understood by a broad audience. The abstract is concise and falls within the typical word limit expected for an academic abstract, summarizing the critical aspects of the study in a brief format.\n\nThe abstract is complete in that it covers the study's objectives, methodology, main findings, and briefly discusses the significance and potential implications of the research. It maintains coherence by logically structuring the content, presenting a clear flow from the research question to the methodology, findings, conclusions, and implications. The abstract is informative and engaging, effectively summarizing the study's contributions and significance to the field, which can attract readers to explore the full

In [25]:
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are an AI researcher who is skilled in academic writing. Your task is to follow the instructions made by the user, to help write better academic papers."},
        {"role": "user", "content": f"Following is research paper content for a method called Constitutional AI. Write a good abstract for the academic paper, based on the content. \nContent: {sample_text}\n Abstract: "},
        {"role": "assistant", "content": f"{initial_response}"},
        {"role": "user", "content": f"Critique Request: Identify if the last response is a good abstract, based on the following principals: \n{constitution}\n Critique: "},
        {"role": "assistant", "content": f"{critique_response}"},
        {"role": "user", "content": f"Revision Request: Based on the critique, incorporate the suggested changes and rewrite the abstract such that it better adheres to the principals of a good abstract described earlier. \nRevision: "},
    ],
)

In [26]:
revised_response = completion.choices[0].message.content

In [27]:
revised_response

"Artificial intelligence (AI) systems are advancing rapidly and require techniques to ensure they remain helpful, honest, and harmless without constant human supervision. This paper introduces Constitutional AI (CAI) as a new approach to training non-evasive and relatively harmless AI assistants. The CAI method utilizes a set of principles or instructions, forming a 'constitution', to guide the AI's behavior without relying on human feedback labels for harm assessment. Through a two-stage training process, CAI outperforms traditional methods by leveraging supervised learning to refine the model's responses based on constitutional principles in the first stage and reinforcement learning with AI feedback in the second stage to enhance the model's performance. The contributions of this study include demonstrating the effectiveness of CAI in training models that exhibit improved harmlessness and providing explanations for objectionable requests. By eliminating the need for human feedback l